Since the original Bechdel website updates very regularly, I wonder if there is a way for me to add the new movies to Kaggle and keep the dataset on Kaggle updated as well. 

I want to UPDATE the dataset, instead of run `bechdel_api.ipynb` again, wait for 7 hours, and risk crashing the website. 

In [27]:
import pandas as pd
import numpy as np

# Load Datasets & Check Updates

In [42]:
# Check the movie list we have now
current_bechdel_detailed = pd.read_csv('../Data/Bechdel_detailed_original.csv')
current_bechdel_detailed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9329 entries, 0 to 9328
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9329 non-null   object 
 1   year         9329 non-null   float64
 2   rating       9329 non-null   float64
 3   dubious      9030 non-null   float64
 4   imdbid       9329 non-null   float64
 5   id           9329 non-null   float64
 6   submitterid  9329 non-null   float64
 7   date         9329 non-null   object 
 8   visible      9329 non-null   float64
dtypes: float64(7), object(2)
memory usage: 656.1+ KB


In [43]:
# Get the latest movie list from the Bechdel website
latest_bechdel = pd.read_json('http://bechdeltest.com/api/v1/getAllMovies')
latest_bechdel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9373 entries, 0 to 9372
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rating  9373 non-null   int64 
 1   title   9373 non-null   object
 2   imdbid  9373 non-null   object
 3   year    9373 non-null   int64 
 4   id      9373 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 366.3+ KB


In [44]:
print('Time to update Kaggle dataset?', not (len(current_bechdel_detailed)==len(latest_bechdel)))

Time to update Kaggle dataset? True


# Compare datasets

In order to know what has been updated over the past few months, I ran Bechdel_api.ipynb again to compare datasets. Again, it took about 7 hours. But hopefully, after this one, I will never have to do it again. 

In [6]:
old_bechdel_detailed = pd.read_csv('./Bechdel_detailed_original.csv')
new_bechdel_detailed = pd.read_csv('./Bechdel_detailed.csv', index_col=0)
new_bechdel_detailed.tail()

,title,year,rating,dubious,imdbid,id,submitterid,date,visible
9368,Love Hard,2021.0,2.0,0.0,10752004.0,10152.0,19735.0,2021-12-05 19:22:20,1.0
9369,Cruella,2021.0,3.0,0.0,3228774.0,9861.0,19231.0,2021-06-01 03:16:58,1.0
9370,West Side Story,2021.0,3.0,0.0,3581652.0,10157.0,19743.0,2021-12-10 03:10:09,1.0
9371,Every Time a Bell Rings,2021.0,3.0,0.0,15943414.0,10158.0,19744.0,2021-12-10 08:03:02,1.0
9372,Single All The Way,2021.0,3.0,0.0,14315756.0,10161.0,19753.0,2021-12-15 10:10:36,1.0


In [38]:
new, rating_changes, dubious_changes = 0, 0, 0
for index, imdbid in new_bechdel_detailed['imdbid'].iteritems():
    new_series = new_bechdel_detailed.loc[index]
    old_series = old_bechdel_detailed[old_bechdel_detailed['imdbid'] == imdbid]
    if imdbid == 35279:
        old_series = old_series.loc[844]
    elif imdbid == 53285:
        old_series = old_series.loc[951] 
    elif imdbid == 86425:
        old_series = old_series.loc[1952] 
    elif imdbid == 117056:
        old_series = old_series.loc[3232] 
    elif imdbid == 2043900:
        old_series = old_series.loc[6470] 
    elif imdbid == 2457282:
        old_series = old_series.loc[7113] 
    elif imdbid == 2180411:
        old_series = old_series.loc[7690] 
    elif imdbid == 2386490:
        old_series = old_series.loc[9098] 
    elif imdbid == 8022928:
        old_series = old_series.loc[9114]
    
    assert not new_series.empty

    if old_series.empty:
        new += 1
        print('NEW')
        print(old_series, new_series)
        print('=============================\n\n')
    else:
        if (old_series == new_series)['rating'].item() is False:
            rating_changes += 1
            print('RATING CHANGES')
            print(old_series, new_series)
            print('=============================\n\n')
        if np.isnan(old_series['dubious'].item()) and np.isnan(new_series['dubious'].item()):
            # np.nan == np.nan returns false
            # so let's not mistaken that as dubious changes
            pass
        elif (old_series == new_series)['dubious'].item() is False:
            dubious_changes += 1
            print('DUBIOUS CHANGES')
            print(old_series, new_series)
            print('=============================\n\n')
    
print('# new movies        :', new)
print('# of rating changes :', rating_changes)
print('# of dubious changes:', dubious_changes)

NEW
Empty DataFrame
Columns: [title, year, rating, dubious, imdbid, id, submitterid, date, visible]
Index: [] title             Flame of My Love
year                          1949
rating                           3
dubious                          0
imdbid                       42031
id                           10135
submitterid                  19707
date           2021-11-21 05:46:58
visible                          1
Name: 669, dtype: object


NEW
Empty DataFrame
Columns: [title, year, rating, dubious, imdbid, id, submitterid, date, visible]
Index: [] title            Blood for dracula
year                          1974
rating                           3
dubious                          0
imdbid                       71233
id                           10128
submitterid                  19697
date           2021-11-15 21:33:09
visible                          1
Name: 1563, dtype: object


NEW
Empty DataFrame
Columns: [title, year, rating, dubious, imdbid, id, submitterid, date, visi

# Update to the latest dataset

`Bechdel.csv` is very easy to update. 2 commands is all it takes. 

`Bechdel_detailed.csv` is much harder. I do not want to wait for 7 hours every time I update the dataset. So, how can I build upon the old dataset and modify it?

What needs to be updated?

- New movies for sure
- Dubious movies might not be dubious anymore (I don't think movies that are not dubious are going to be converted back to dubious)
- Ratings might change

In [48]:
latest_bechdel

,rating,title,imdbid,year,id
0,0,Passage de Venus,3155794,1874,9602
1,0,La Rosace Magique,14495706,1877,9804
2,0,Sallie Gardner at a Gallop,2221420,1878,9603
3,0,Le singe musicien,12592084,1878,9806
4,0,Athlete Swinging a Pick,7816420,1881,9816
...,...,...,...,...,...
9368,2,Love Hard,10752004,2021,10152
9369,3,Cruella,,2021,10155
9370,3,West Side Story,3581652,2021,10157
9371,3,Every Time a Bell Rings,15943414,2021,10158


In [37]:
for index, imdbid in latest_bechdel['imdbid'].iteritems():
    latest_bechdel_row = latest_bechdel.loc[index]
    

0.0